# Doodles and notes - nothing important

In [ ]:
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import network_diffusion as nd
import networkx as nx
import pandas as pd
import itertools

%load_ext autoreload
%autoreload 2

In [ ]:
layer_1 = nx.Graph(
    (
        [1, 4], [2, 4], [2, 3], [3, 4], [3, 5],
        [3, 6], [4, 5], [7, 9], [8, 9], [8, 10]
    )
)

layer_2 = nx.Graph(
    (
        [1, 2], [2, 7], [2, 11], [4, 5], [4, 6],
        [5, 11], [6, 10], [7, 9], [8, 9], [8, 10],
        [10, 11]
    )
)

layer_3 = nx.Graph(
    (
        [1, 4], [2, 6], [2, 9], [3, 4], [3, 5],
        [4, 5], [5, 6], [5, 11], [6, 9], [7, 9], [10, 11],
    )
)

net = nd.MultilayerNetwork.from_nx_layers([layer_1, layer_2, layer_3], ["l1", "l2", "l3"])

## Neighbourhood size

In [ ]:
from typing import Dict

from misc.loader import *
from network_diffusion.mln.mlnetwork import MLNetworkActor, MultilayerNetwork

ns = nd.mln.functions.neighbourhood_size
nsh = nd.mln.functions._ns_helper

def ns_old(net: MultilayerNetwork) -> Dict[MLNetworkActor, int]:
    """Return neighbourhood sizes of all actors from the network."""
    neighbourhood_sizes: Dict[MLNetworkActor, int] = {}
    for actor in net.get_actors():
        a_neighbours: Set[Any] = set()
        for l_name in actor.layers:
            a_neighbours = a_neighbours.union(
                set(net.layers[l_name].adj[actor.actor_id].keys())
            )
        neighbourhood_sizes[actor] = len(a_neighbours)
    return neighbourhood_sizes

nets = [
    net,
    get_aucs_network(), 
    get_ckm_physicians_network(),
    get_eu_transportation_network(),
    get_lazega_network(),
    get_er2_network(),
    get_er3_network(),
    get_er5_network(),
    get_sf2_network(),
    get_sf3_network(),
    get_sf5_network(),
]

def ravel_rank(rank):
    return {a.actor_id: v for a, v in rank.items()}

for n in nets:
    print(ravel_rank(ns(net, 1)) == ravel_rank(ns_old(net)))

## K-shell for single layer network

In [ ]:

g = nx.Graph()
g.add_edges_from(
    [(1, 2), (1, 9), (3, 13), (4, 6),
    (5, 6), (5, 7), (5, 8), (5, 9),
    (5, 10), (5, 11), (5, 12), (10, 12),
    (10, 13), (11, 14), (12, 14),
    (12, 15), (13, 14), (13, 15),
    (13, 17), (14, 15), (15, 16)]
)
nx.draw(g, with_labels=True)

In [ ]:
gg = nx.core.k_shell(g, 2)
nx.draw(gg, with_labels=True)

In [ ]:
net = nd.MultilayerNetwork.load_mlx(file_path="data/aucs.mpx")
fb = net.layers["facebook"]

In [ ]:
gg = fb.copy()
fig, axes = plt.subplots(nrows=3, ncols=3)
row = -1
col = -1

for k in range(9):

    row += 1
    if row % 3 == 0:
        row = 0
        col += 1

    axis = axes[col][row]
    axis.set_title(f"K={k}")

    ggg = nx.k_shell(gg, k=k)
    
    nx.draw(ggg, with_labels=True, ax=axis)
    print(k, [[node, nx.degree(gg, node)] for node in ggg.nodes()])

    if len(list(gg.nodes())) == 0:
        break

fig.set_size_inches(18.5, 10.5)
fig.tight_layout()

In [ ]:
nx.draw(sh:=nx.k_shell(fb), with_labels=True)
print(k, [[node, nx.degree(fb, node)] for node in sh.nodes()])

In [ ]:
nx.draw(fb, with_labels=True)

In [ ]:
def create_k_shell_list(g: nx.Graph):
    ksh_deepest_nodes = set(nx.k_shell(g).nodes())
    shell_ranking = {}
    k = 0

    while True:
        ksh_nodes = set(nx.k_shell(g, k=k).nodes())
        shell_ranking[k] = ksh_nodes
        if ksh_nodes == ksh_deepest_nodes:
            break
        k += 1
    
    return shell_ranking

In [ ]:
rl = create_k_shell_list(fb)

In [ ]:
print(sorted(rl)[::-1])

In [ ]:
rl

In [ ]:
ll = [list(rl[k]) for k in sorted(rl)[::-1]]
import itertools

In [ ]:
lll = [n for cohort in sorted(rl)[::-1] for n in rl[cohort]]

In [ ]:
lll

## Degree centrality

In [ ]:
print(net.get_actors_num())
for actor in net.get_actors():
    print(actor)

In [ ]:
selector = nd.seeding.PageRankSeedSelector()
ranking = selector(net, actorwise=True)
print([r.actor_id for r in ranking])

In [ ]:
nx.pagerank(net.layers["l1"])

In [ ]:
my_dict = { 'num6': 6, "34": 2, 'num2': 2, 'num4': 4, 'num1': 1, 'num5': 5}
sortedDict = sorted(my_dict.values())

In [ ]:
sortedDict

In [ ]:
import numpy as np

In [ ]:
seeding_budgets = np.arange(0, 101, 1)

In [ ]:
s = [(a, 100 - a) for a in np.arange(0, 101, 1)]

In [ ]:
print(s)

In [ ]:
z = np.logspace(-2, 0, num=100)

In [ ]:
print(z)

## K-shell for multi layer network

In [ ]:
from network_diffusion.mln.functions import k_shell_mln, degree, neighbourhood_size

net ## net from degree centrality
degrees = degree(net)
sorted(
        [*degrees.keys()],
        key=lambda x: degree(net)[x],
        reverse=True,
)

In [ ]:
neighbourhood_size(net)

In [ ]:

net.layers["l1"].nodes[5]["status"] = "test"
k_net = k_shell_mln(net)
print(degree(k_net))
print(k_net.layers["l1"].nodes[5]["status"])

In [ ]:
ksh_deepest_nodes = set(k_shell_mln(net).get_actors())
shell_ranking = {}
k = 0

# iterate until deepest shell is achieved
while True:

    # compute k-shell cohort
    ksh_nodes = k_shell_mln(net, k=k).get_actors()

    # sort it according to degree in the graph
    shell_ranking[k] = sorted(
        ksh_nodes,
        key=lambda x: degree(net)[x],
        reverse=True,
    )

    # if the deepest shell is reached breake, othrwise increase k
    if set(ksh_nodes) == ksh_deepest_nodes:
        break
    k += 1

shell_ranking

In [ ]:
ksh_selector = nd.seeding.KShellSeedSelector()
ksh_selector.actorwise(net)

## Pagerank for MLNet

In [ ]:
l1_net = net.layers["l1"]
nx.pagerank(l1_net)

squeezed_net = nd.mln.functions.squeeze_by_neighbourhood(net)
raw_dict = nx.pagerank(squeezed_net)
sorted_dict = sorted(
    raw_dict, key=lambda x: raw_dict[x], reverse=True
)

# for k, v in raw_dict.items():
#     print(k.actor_id, v)
print(sorted_dict)


In [ ]:
nd.seeding.PageRankMLNSeedSelector().actorwise(net)

In [ ]:
from loader import get_sf2_network
sf2 = get_sf2_network()
net_len = sf2.get_actors_num()

In [ ]:
rank_len = len(nd.seeding.PageRankMLNSeedSelector().actorwise(sf2))
sqnet_len = len(nd.mln.functions.squeeze_by_neighbourhood(sf2))
print(net_len, sqnet_len, rank_len)

## Voterank for MLNet

In [ ]:
from network_diffusion.mln.functions import voterank_actorwise
from network_diffusion.mln.actor import MLNetworkActor

# [[*l_graph.edges()] for l_graph in net.layers.values()]

for _, nbr in net.layers["l1"].edges(1):
    print(_, nbr)
    # vote_rank[nbr][1] -= 1 / avgDegree
    # vote_rank[nbr][1] = max(vote_rank[nbr][1], 0)

# n2 = nd.MultilayerNetwork.from_nx_layer(net.layers["l3"].copy(), layer_names=["A", "B", "C"])

# net.is_directed()
# print(voterank_actorwise(n2))
print(voterank_actorwise(net))
# print(nx.voterank(n2.layers["C"]))

In [ ]:
for a, b in net.get_links(10):
    print(f"a: {a.actor_id}, b: {b.actor_id}")


In [ ]:
nd.seeding.VoteRankMLNSeedSelector().actorwise(net)

## reading from .edges

In [ ]:
def get_ckm_network(path):
    df = pd.read_csv(path, names=["node_1", "node_2", "layer"])
    net_dict = {l_name: nx.Graph() for l_name in [*df["layer"].unique()]}
    for _, row in df.iterrows():
        net_dict[row["layer"]].add_edge(row["node_1"], row["node_2"])
    return nd.MultilayerNetwork.load_layers_nx(
        layer_names=[*net_dict.keys()], network_list=[*net_dict.values()]
    )

In [ ]:
ckm = "data/CKM-Physicians-Innovation_4NoNature.edges"
eu_transportation = "data/EUAirTransportation_multiplex_4NoNature.edges"
lazega = "data/Lazega-Law-Firm_4NoNatureNoLoops.edges"



In [ ]:
net = get_ckm_network(lazega)
print(net.get_actors_num())
print(_ := net.get_nodes_num(), sum(_.values()), len(set(_.keys())))

In [ ]:
def get_aucs_network(file_path):
    return nd.MultilayerNetwork.load_mlx(file_path)

net = get_aucs_network(file_path="data/erererererNoLoops.mpx")
print(net.get_actors_num())
print(_ := net.get_nodes_num(), sum(_.values()), len(set(_.keys())), sum([l.number_of_edges() for l in net.layers.values()]))

## Classical metrics aggregation

In [ ]:
ranking = nd.seeding.VoteRankSeedSelector().nodewise(net=net)

print(ranking)
print({l_name: len(l_graph) for l_name, l_graph in net.layers.items()})

aggregated_ranking = nd.seeding.VoteRankSeedSelector().actorwise(net=net)

print(aggregated_ranking)